In [ ]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [ ]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [ ]:
class NeuralNetwork(nn.Module) :
    def __init__ (self) :
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512), 
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork()

# Setting hyperparameter
- ephochs : 전체 데이터 셋에 대해 학습을 완료한 상태 
- batch : 한번에 주어지는 데이터 샘플 사이즈
- learning rate

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Add an optimization loop
- train loop : hyperparameter를 찾으려고 노력 
- valid/test loop

## Add a loss function
- nn.MSELoss : 회귀 모델일 때 
- nn.NLLoss : 분류 모델일 때 
- nn.CrossEntropyLoss : 다중 분류, nn.LogSoftmax와 nn.NLLoss가 포함되어 있다. 

In [ ]:
loss_fn = nn.CrossEntropyLoss()

## Optimization pass

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Backpropagation
- optimizer.zero_grad()
> 역전파 단계를 실행하기 전에 변화도(gradient)를 0으로 재설정
- loss.backward()
> 역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도 계산
- optimizer.step()
> 변화도를 계산한 뒤에 `optimizer.step()`을 호출하여 역전파 단계에서 수집된 변화도로 매개변수 조정

# Full implementation

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer) :
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader) :
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    # test에서는 학습을 하면 안 되기 때문에 no_grad 사용 
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            # 손실값과 정답 개수를 누적 
            test_loss += loss_fn(pred, y).item()
            # pred 값이 정답값과 일치하는 개수를 카운트 
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs) :
    print(f'Epoch {t+1}\n-----------------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-----------------------------
loss: 2.304868  [    0/60000]
loss: 2.290684  [ 6400/60000]
loss: 2.277122  [12800/60000]
loss: 2.267117  [19200/60000]
loss: 2.253782  [25600/60000]
loss: 2.251516  [32000/60000]
loss: 2.228769  [38400/60000]
loss: 2.212580  [44800/60000]
loss: 2.218316  [51200/60000]
loss: 2.207201  [57600/60000]
Test Error: 
 Accuracy: 53.4%, Avg loss: 0.034378 

Epoch 2
-----------------------------
loss: 2.200376  [    0/60000]
loss: 2.181355  [ 6400/60000]
loss: 2.151266  [12800/60000]
loss: 2.156868  [19200/60000]
loss: 2.098808  [25600/60000]
loss: 2.115909  [32000/60000]
loss: 2.070491  [38400/60000]
loss: 2.038041  [44800/60000]
loss: 2.067286  [51200/60000]
loss: 2.051897  [57600/60000]
Test Error: 
 Accuracy: 54.0%, Avg loss: 0.031672 

Epoch 3
-----------------------------
loss: 2.038775  [    0/60000]
loss: 2.001968  [ 6400/60000]
loss: 1.947967  [12800/60000]
loss: 1.975177  [19200/60000]
loss: 1.858221  [25600/60000]
loss: 1.906765  [32000/60000]
lo

# Saving models

In [ ]:
# 각각의 weight와 bias(parameter)를 dictionary 형태로 저장
torch.save(model.state_dict(), 'data/model.pth')
print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


# Loading models

In [ ]:
# 모델의 틀을 먼저 불러오고, 학습한 파라미터 값을 적용 
model = NeuralNetwork()
model.load_state_dict(torch.load('data/model.pth'))

<All keys matched successfully>

# Predictions

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted : "{predicted}", Actual: "{actual}')

Predicted : "Ankle boot", Actual: "Ankle boot
